In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 1. Data Preparation (same as before)
df = pd.read_csv('dataset.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

features = ['temperature', 'humidity', 'wind_speed', 'power']
df = df[features]

scalers = {}
for column in df.columns:
    scaler = MinMaxScaler()
    df[column] = scaler.fit_transform(df[[column]])
    scalers['scaler_' + column] = scaler

n_past = 10
n_future = 5
n_features = len(features)

X = []
y = []
for i in range(n_past, len(df) - n_future + 1):
    X.append(df.iloc[i - n_past:i].values)
    y.append(df.iloc[i:i + n_future].values)

X = np.array(X)
y = np.array(y)

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# 2. VAE-LSTM Model (Without separate Encoder/Decoder)
inputs = tf.keras.layers.Input(shape=(n_past, n_features))

# LSTM layers (acting as the core of the VAE)
lstm_1 = tf.keras.layers.LSTM(16, return_sequences=True)(inputs)
lstm_2 = tf.keras.layers.LSTM(8)(lstm_1)  # Reduced LSTM units

# Variational layers (applied directly to the LSTM output)
z_mean = tf.keras.layers.Dense(2, name="z_mean")(lstm_2)  # Reduced latent dimension
z_log_var = tf.keras.layers.Dense(2, name="z_log_var")(lstm_2)  # Reduced latent dimension

class KLDivergenceLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(KLDivergenceLayer, self).__init__(**kwargs)

    def call(self, inputs):
        z_mean, z_log_var = inputs
        kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
        self.add_loss(kl_loss)
        return z_mean

z_mean = KLDivergenceLayer()([z_mean, z_log_var])

def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = tf.keras.layers.Lambda(sampling, name="z")([z_mean, z_log_var])


# Repeat the latent vector to match the desired output sequence length
repeated_z = tf.keras.layers.RepeatVector(n_future)(z)

# LSTM layers to process the repeated z
lstm_3 = tf.keras.layers.LSTM(16, return_sequences=True)(repeated_z)
lstm_4 = tf.keras.layers.LSTM(8, return_sequences=False)(lstm_3)

# Output layer
outputs = tf.keras.layers.Dense(n_features * n_future)(lstm_4)
outputs = tf.keras.layers.Reshape([n_future, n_features])(outputs)

vae_lstm = tf.keras.models.Model(inputs=inputs, outputs=outputs, name="vae_lstm")



# 3. Compilation and Training (same as before)
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

vae_lstm.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_vae = vae_lstm.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=4, verbose=1, callbacks=[reduce_lr, early_stopping])

# 4. Prediction and Inverse Scaling (same as before)
pred_vae = vae_lstm.predict(X_test)

for index, i in enumerate(features):
    scaler = scalers['scaler_' + i]
    pred_vae[:, :, index] = scaler.inverse_transform(pred_vae[:, :, index])
    y_train[:, :, index] = scaler.inverse_transform(y_train[:, :, index])
    y_test[:, :, index] = scaler.inverse_transform(y_test[:, :, index])

# 5. Evaluation (same as before)
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

y_true_all = y_test.flatten()
y_pred_vae_all = pred_vae.flatten()

mae_vae = mean_absolute_error(y_true_all, y_pred_vae_all)
mse_vae = mean_squared_error(y_true_all, y_pred_vae_all)
rmse_vae = np.sqrt(mse_vae)
mape_vae = mean_absolute_percentage_error(y_true_all, y_pred_vae_all)
r2_vae = r2_score(y_true_all, y_pred_vae_all)

results_df = pd.DataFrame({
    "Model": ["VAE-LSTM (No Encoder/Decoder)"],
    "MAE": [mae_vae],
    "MSE": [mse_vae],
    "RMSE": [rmse_vae],
    "MAPE": [mape_vae],
    "R2 Score": [r2_vae]
})

print(results_df)

y_test_2d = y_test.reshape(-1, y_test.shape[-1])
pred_vae_2d = pred_vae.reshape(-1, pred_vae.shape[-1])

y_var = np.var(y_test_2d)
normalized_mse_vae = mean_squared_error(y_test_2d, pred_vae_2d) / y_var

print("Normalized MSE VAE:", normalized_mse_vae)

# Predict next 10 days (same as before)
X_test_last = X_test[-1:].reshape(1, n_past, n_features)
pred_next_10_days_vae = vae_lstm.predict(X_test_last)

for index, i in enumerate(features):
    scaler = scalers['scaler_' + i]
    pred_next_10_days_vae[:, :, index] = scaler.inverse_transform(pred_next_10_days_vae[:, :, index])

print("Predicted next 10 days (VAE-LSTM without Encoder/Decoder):", pred_next_10_days_vae)